In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from numpy import linalg as LA

%matplotlib inline



n=100
q=1000
ratio=q/n
#print('The ratio is', ratio)

#sparse level
#sparse_level=0.2 #fixed
#sparse_level=0.6
#sparse_level=0.9


sensitivity_average=[]
specificity_average=[]
accuracy_average=[]

sparce_vector=np.linspace(0.1,0.90,10)

for sparse in range(10):
    sensitivity_vec=[]
    specificity_vec=[]
    accuracy_vec=[]
    
    for j in range(10):

        sparse_level=sparce_vector[sparse]

        beta=np.random.normal(0, 1, q)
        zero_one = np.random.choice([0, 1], size=q, p=[sparse_level, 1-sparse_level])
        beta = beta*zero_one

        X=np.random.normal(0, 1, (n,q))
        SNR=5 # fixed
        gamma=SNR
        sigma = np.sqrt(LA.norm(X.dot(beta))**2/(n-1))/gamma
        epsilon=np.random.normal(0, sigma, n)

        y=X.dot(beta)+epsilon
        lasso = Lasso(max_iter = 1000000, normalize = True)
        coefs = []

        #generating an array of alpha (lambda) values ranging from very big to very small
        alphas = 10**np.linspace(10,-2,100)*0.5

        for a in alphas:
            lasso.set_params(alpha=a)
            lasso.fit(X, y)
            coefs.append(lasso.coef_)


        ########################################

        # Split data into training and test sets
        X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        #perform 10-fold cross-validation to choose the best lambda
        lassocv = LassoCV(alphas = None, cv = 10, max_iter = 1000000, normalize = True)
        lassocv.fit(X_train, y_train)

        #refit the model
        lasso.set_params(alpha=lassocv.alpha_)
        lasso.fit(X_train, y_train)
        #print('The optimal lambda is:',lassocv.alpha_)

        #compute the associated test error:
        #y_pred=lasso.predict(X_test)
        #meanSqError=mean_squared_error(y_test, y_pred)
        #print('The mean squared error is:', meanSqError)

        #plt.axvline(x=lassocv.alpha_,color='red',linestyle='dotted')
        #plt.title('Lasso with optimal lambda: %s' %(round(lassocv.alpha_,3)))


        true_positive=0
        false_positive=0
        true_negative=0
        false_negative=0

        beta_hat=lasso.coef_

        for i in range(q):
            if beta_hat[i]!=0:
                if beta[i]!=0:
                    true_positive=true_positive+1
                else:
                    false_positive=false_positive+1
            else:
                if beta[i]==0:
                    true_negative=true_negative+1
                else:
                    false_negative=false_negative+1


        #print('TP=',true_positive)
        #print('FP=',false_positive)
        #print('TN=',true_negative)
        #print('FN=',false_negative)

        #sensitivity
        sensitivity=true_positive/(true_positive+false_negative)
        sensitivity_vec.append(sensitivity)

        #specificity
        specificity=true_negative/(true_negative+false_positive)
        specificity_vec.append(specificity)

        #accuracy
        accuracy=(true_positive+true_negative)/(true_positive+true_negative+false_negative+false_positive)
        accuracy_vec.append(accuracy)

    sensitivity_average.append(np.mean(sensitivity_vec))
    specificity_average.append(np.mean(specificity_vec))
    accuracy_average.append(np.mean(accuracy_vec))


In [ ]:
x=sparce_vector
y_accuracy=np.array(accuracy_average)
plt.scatter(x,y_accuracy)

z = np.polyfit(x, y_accuracy, 3)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

xmin, xmax, ymin, ymax = plt.axis([0, 1, 0, 1])
plt.title("SNR= %s, p/n= %s" %(SNR, round(ratio,2)))
plt.xlabel('Sparsity level')
plt.ylabel('Accuracy')



In [ ]:
y_sensitivity=np.array(sensitivity_average)
plt.scatter(x,y_sensitivity)

z = np.polyfit(x, y_sensitivity, 4)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

xmin, xmax, ymin, ymax = plt.axis([0, 1, 0, 1])
plt.title("SNR= %s, p/n= %s" %(SNR, round(ratio,2)))
plt.xlabel('Sparsity level')
plt.ylabel('Sensitivity')

In [ ]:
y_specificity=np.array(specificity_average)
plt.scatter(x,y_specificity)

z = np.polyfit(x, y_specificity, 5)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

xmin, xmax, ymin, ymax = plt.axis([0, 1, 0, 1])
plt.title("SNR= %s, p/n= %s" %(SNR, round(ratio,2)))
plt.xlabel('Sparsity level')
plt.ylabel('Specificity')

In [ ]:
x=sparce_vector
y_accuracy=np.array(accuracy_average)
y_sensitivity=np.array(sensitivity_average)
y_specificity=np.array(specificity_average)
xmin, xmax, ymin, ymax = plt.axis([0, 1, 0, 1])
plt.scatter(x,y_accuracy,label="Accuracy")
plt.scatter(x,y_sensitivity,label="Sensitivity")
plt.scatter(x,y_specificity,label="Specificity")


z_accuracy = np.polyfit(x, y_accuracy, 3)
p_accuracy = np.poly1d(z_accuracy)

z_sensitivity = np.polyfit(x, y_sensitivity, 4)
p_sensitivity = np.poly1d(z_sensitivity)

z_specificity = np.polyfit(x, y_specificity, 5)
p_specificity = np.poly1d(z_specificity)
plt.plot(x,p_accuracy(x),"--")
plt.plot(x,p_sensitivity(x),"--")
plt.plot(x,p_specificity(x),"--")

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.title("SNR= %s, p/n= %s" %(SNR, round(ratio,2)))
plt.xlabel('Sparsity level')
plt.ylabel('Ratio')

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
